In [2]:
import torch
import torch.optim as optim
import numpy as np


n = 2
k = 2


quantities = [[10, 20, 10], [60, 40, 100], [100, 150, 190], [5, 1, 2], [1000, 1500, 2000]]  # Example initial quantities for each AMM
initial_quantities = torch.tensor(quantities, requires_grad=False)

# Define Delta for the first n-1 AMMs only, since the last one will be derived
Delta_incomplete = torch.zeros((n-1, k), requires_grad=True)

def get_Delta(Delta_incomplete):
    assert Delta_incomplete.shape == (n-1, k), f"Delta_incomplete should have shape {(n-1, k)}"
    last_row_adjustment = -torch.sum(Delta_incomplete, axis=0, keepdims=True)
    # Combine adjustments to get the full Delta matrix
    return torch.cat((Delta_incomplete, last_row_adjustment), axis=0)

# Objective function considering the new Delta definition
def utilities(Delta):
    assert Delta.shape == (n, k), f"Delta should have shape {(n, k)}"
    # Calculate adjustments for the last AMM as the negation of the sum of the others
    new_quantities = initial_quantities + Delta
    products = new_quantities.prod(dim=1)  # Product of token quantities in each AMM
    return products

# get the gradient of utility w.r.t Delta
def get_gradient(Delta):
    assert Delta.shape == (n, k), f"Delta should have shape {(n, k)}"
    utilities_per_amm = utilities(Delta)
    gradients_per_amm = []
    for i in range(n):
        if i < n - 1:
            grad = torch.autograd.grad(utilities_per_amm[i], Delta_incomplete[i], retain_graph=True)[0]
        else:
            # For the last AMM, compute gradient with respect to the negation of the sum of previous Deltas
            grad = torch.autograd.grad(utilities_per_amm[i], Delta_incomplete, retain_graph=True)[0]
            grad = -torch.sum(grad, dim=0)  # Sum and negate to get the gradient for the last row
        gradients_per_amm.append(grad)

    return torch.stack(gradients_per_amm)
    # this has shape (n, k)

def cosine_utility(cosine_similarities):
    return cosine_similarities.sum()

def cosine_difference_loss(cosine_similarities):
    return cosine_similarities.var()

def cosine_similarities(grads, Delta):
    assert grads.shape == (n, k), f"grads should have shape {(n, k)}"
    assert Delta.shape == (n, k), f"Delta should have shape {(n, k)}"
    cosines = []
    for i in range(n):
        grad = grads[i]
        cos_times_v = torch.dot(grad, Delta[i]) / (torch.linalg.vector_norm(grad))
        cosines.append(cos_times_v)
    return torch.stack(cosines)

# Define the optimizer
optimizer = optim.Adam([Delta_incomplete], lr=0.01)

# Training loop
for i in range(1000):
    optimizer.zero_grad()
    Delta = get_Delta(Delta_incomplete)
    cos_similarities = cosine_similarities(get_gradient(Delta), Delta)
    cos_utility = cosine_utility(cos_similarities)
    cos_loss = cosine_difference_loss(cos_similarities)
    relu_loss = torch.sum(torch.relu(-initial_quantities - get_Delta(Delta_incomplete)))
    loss = 0
    loss -= cos_utility
    loss += cos_loss
    loss += 1000*relu_loss
    print(f"Cos utility: {cos_utility} | Cos loss: {cos_loss} | ReLU loss: {relu_loss}")
    loss.backward()
    optimizer.step()
    # if i % 100 == 0:
    #     print(f"Iteration {i}, loss: {loss.item()}")

    # if i % 100 == 0:
    #     print(f"Utility: {utility}")
    #     print(f"Delta: {Delta}")
    #     grads = get_gradient(Delta)
    #     print(f"Gradients: {grads}")
    #     cosines = cosine_similarities(grads, Delta)
    #     print(f"Cosine similarities: {cosines}")
    #     print("")

final_delta = get_Delta(Delta_incomplete).detach()
print(f"Final Delta: {final_delta}")
print(f"Final Utility: {utilities(final_delta)}")
print(f"Final Gradient: {get_gradient(final_delta)}")
print(f"Final Cosine similarities: {cosine_similarities(get_gradient(final_delta), final_delta)}")
print(f"Final Cosine utility: {cosine_utility(cosine_similarities(get_gradient(final_delta), final_delta))}")
print(f"Final Cosine loss: {cosine_difference_loss(cosine_similarities(get_gradient(final_delta), final_delta))}")
print(f"Final ReLU loss: {torch.sum(torch.relu(-initial_quantities - final_delta))}")

print(f"Final balances: {initial_quantities + final_delta}")


RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1